# Pruebas de Hipótesis para 1 y 2 Muestras

## Objetivos
- Entender el proceso de pruebas de hipótesis
- Realizar prueba t para una muestra
- Realizar prueba t para dos muestras independientes
- Realizar prueba t pareada
- Interpretar p-values y tomar decisiones estadísticas

---

## 1. Preparación

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
%matplotlib inline

# Semilla para reproducibilidad
np.random.seed(42)

# Cargar datos
df = pd.read_csv('../datos/ejemplo_satisfaccion_clientes.csv')
print(f"Dataset cargado: {df.shape[0]} registros")

## 2. Recordatorio: Pasos de una Prueba de Hipótesis

1. **Plantear hipótesis** (H₀ y H₁)
2. **Seleccionar nivel de significancia** (α, típicamente 0.05)
3. **Calcular estadístico de prueba** (t, Z, etc.)
4. **Calcular p-value**
5. **Tomar decisión:** Si p-value < α → Rechazar H₀

---

## 3. Prueba t para UNA Muestra

### Contexto de Negocio:
La ONG afirma que su satisfacción promedio es de **8.0/10**. ¿Los datos respaldan esta afirmación?

### Hipótesis:
- **H₀:** μ = 8.0 (La media poblacional es 8)
- **H₁:** μ ≠ 8.0 (La media poblacional es diferente de 8) [Bilateral]

In [ ]:
# Paso 1: Plantear hipótesis
mu_0 = 8.0  # Valor hipotético
alpha = 0.05  # Nivel de significancia

print("="*60)
print("PRUEBA T PARA UNA MUESTRA")
print("="*60)
print(f"H₀: μ = {mu_0}")
print(f"H₁: μ ≠ {mu_0} (prueba bilateral)")
print(f"α = {alpha}")
print("="*60)

In [ ]:
# Paso 2: Calcular estadísticos de la muestra
satisfaccion = df['satisfaccion']
n = len(satisfaccion)
media_muestral = satisfaccion.mean()
desv_std_muestral = satisfaccion.std()

print("\nEstadísticos de la muestra:")
print(f"n = {n}")
print(f"Media muestral (x̄) = {media_muestral:.4f}")
print(f"Desviación estándar (s) = {desv_std_muestral:.4f}")

In [ ]:
# Paso 3: Realizar prueba t
t_statistic, p_value = stats.ttest_1samp(satisfaccion, mu_0)

print("\n" + "="*60)
print("RESULTADOS DE LA PRUEBA")
print("="*60)
print(f"Estadístico t: {t_statistic:.4f}")
print(f"p-value: {p_value:.4f}")
print("="*60)

In [ ]:
# Paso 4: Decisión
print("\n📊 DECISIÓN:")
if p_value < alpha:
    print(f"   p-value ({p_value:.4f}) < α ({alpha})")
    print("   ✗ RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: Hay evidencia estadística suficiente para afirmar que")
    print(f"   la satisfacción promedio ES DIFERENTE de {mu_0}")
    print(f"   (La satisfacción promedio real es aproximadamente {media_muestral:.2f})")
else:
    print(f"   p-value ({p_value:.4f}) ≥ α ({alpha})")
    print("   ✓ NO RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: No hay evidencia suficiente para rechazar que")
    print(f"   la satisfacción promedio sea {mu_0}")

In [ ]:
# Visualización
plt.figure(figsize=(12, 6))
plt.hist(satisfaccion, bins=15, color='skyblue', edgecolor='black', alpha=0.7, density=True)
plt.axvline(media_muestral, color='blue', linestyle='--', linewidth=2, label=f'Media muestral = {media_muestral:.2f}')
plt.axvline(mu_0, color='red', linestyle='--', linewidth=2, label=f'μ₀ = {mu_0}')

plt.title('Distribución de Satisfacción vs Hipótesis Nula', fontsize=14, fontweight='bold')
plt.xlabel('Satisfacción', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Prueba t para DOS Muestras Independientes

### Contexto de Negocio:
¿Existe diferencia significativa en la satisfacción entre el **Área Norte** y el **Área Sur**?

### Hipótesis:
- **H₀:** μ₁ = μ₂ (Las medias son iguales)
- **H₁:** μ₁ ≠ μ₂ (Las medias son diferentes) [Bilateral]

In [ ]:
# Filtrar datos por área
area_norte = df[df['area'] == 'Norte']['satisfaccion']
area_sur = df[df['area'] == 'Sur']['satisfaccion']

print("="*60)
print("PRUEBA T PARA DOS MUESTRAS INDEPENDIENTES")
print("="*60)
print(f"H₀: μ_Norte = μ_Sur")
print(f"H₁: μ_Norte ≠ μ_Sur (prueba bilateral)")
print(f"α = {alpha}")
print("="*60)

print(f"\nEstadísticos descriptivos:")
print(f"\nNORTE:")
print(f"  n = {len(area_norte)}")
print(f"  Media = {area_norte.mean():.4f}")
print(f"  Desv. Std = {area_norte.std():.4f}")

print(f"\nSUR:")
print(f"  n = {len(area_sur)}")
print(f"  Media = {area_sur.mean():.4f}")
print(f"  Desv. Std = {area_sur.std():.4f}")

In [ ]:
# Verificar supuesto: Homogeneidad de varianzas (Test de Levene)
stat_levene, p_levene = stats.levene(area_norte, area_sur)

print("\n" + "="*60)
print("VERIFICACIÓN DE SUPUESTOS")
print("="*60)
print("Test de Levene (homogeneidad de varianzas):")
print(f"  Estadístico: {stat_levene:.4f}")
print(f"  p-value: {p_levene:.4f}")

if p_levene > 0.05:
    print("  ✓ Las varianzas son homogéneas (usar prueba t estándar)")
    equal_var = True
else:
    print("  ✗ Las varianzas NO son homogéneas (usar Welch's t-test)")
    equal_var = False

In [ ]:
# Realizar prueba t para dos muestras
t_stat, p_val = stats.ttest_ind(area_norte, area_sur, equal_var=equal_var)

print("\n" + "="*60)
print("RESULTADOS DE LA PRUEBA")
print("="*60)
print(f"Estadístico t: {t_stat:.4f}")
print(f"p-value: {p_val:.4f}")
print("="*60)

In [ ]:
# Decisión
print("\n📊 DECISIÓN:")
if p_val < alpha:
    print(f"   p-value ({p_val:.4f}) < α ({alpha})")
    print("   ✗ RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: HAY diferencia significativa en satisfacción entre áreas")
    print(f"   Norte: {area_norte.mean():.2f} vs Sur: {area_sur.mean():.2f}")
    
    if area_norte.mean() > area_sur.mean():
        print(f"   → El Área Norte tiene MAYOR satisfacción")
    else:
        print(f"   → El Área Sur tiene MAYOR satisfacción")
else:
    print(f"   p-value ({p_val:.4f}) ≥ α ({alpha})")
    print("   ✓ NO RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: NO hay evidencia de diferencia significativa")
    print(f"   Las áreas Norte y Sur tienen satisfacción similar")

In [ ]:
# Calcular tamaño del efecto (Cohen's d)
def cohen_d(group1, group2):
    """Calcula el tamaño del efecto de Cohen"""
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std

d = cohen_d(area_norte, area_sur)

print(f"\n📏 TAMAÑO DEL EFECTO (Cohen's d): {d:.4f}")
if abs(d) < 0.2:
    print("   → Efecto PEQUEÑO")
elif abs(d) < 0.5:
    print("   → Efecto PEQUEÑO a MEDIANO")
elif abs(d) < 0.8:
    print("   → Efecto MEDIANO")
else:
    print("   → Efecto GRANDE")

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplots
axes[0].boxplot([area_norte, area_sur], labels=['Norte', 'Sur'], patch_artist=True,
                boxprops=dict(facecolor='lightblue'),
                medianprops=dict(color='red', linewidth=2))
axes[0].set_ylabel('Satisfacción', fontsize=12)
axes[0].set_title('Comparación de Satisfacción (Boxplot)', fontsize=13, fontweight='bold')
axes[0].grid(alpha=0.3, axis='y')

# Histogramas superpuestos
axes[1].hist(area_norte, bins=10, alpha=0.5, label=f'Norte (μ={area_norte.mean():.2f})', color='blue', edgecolor='black')
axes[1].hist(area_sur, bins=10, alpha=0.5, label=f'Sur (μ={area_sur.mean():.2f})', color='orange', edgecolor='black')
axes[1].set_xlabel('Satisfacción', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribuciones Superpuestas', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Prueba t para Muestras Pareadas

### Contexto de Negocio:
La ONG implementó una mejora en el servicio. Tenemos satisfacción **antes** y **después** de la intervención para los mismos beneficiarios.

### Hipótesis:
- **H₀:** μ_diferencia = 0 (No hay cambio)
- **H₁:** μ_diferencia > 0 (Hubo mejora) [Unilateral derecha]

In [ ]:
# Simular datos pareados (antes y después de intervención)
# Para este ejemplo, creamos datos sintéticos
np.random.seed(42)
n_pareado = 30

# Satisfacción antes
antes = np.random.normal(loc=7.0, scale=1.2, size=n_pareado)
antes = np.clip(antes, 1, 10)  # Limitar entre 1-10

# Satisfacción después (con mejora promedio de 0.8 puntos)
despues = antes + np.random.normal(loc=0.8, scale=0.5, size=n_pareado)
despues = np.clip(despues, 1, 10)

# Crear DataFrame
df_pareado = pd.DataFrame({
    'id': range(1, n_pareado+1),
    'antes': antes,
    'despues': despues,
    'diferencia': despues - antes
})

print("Primeros 10 registros:")
print(df_pareado.head(10))

In [ ]:
print("="*60)
print("PRUEBA T PAREADA")
print("="*60)
print(f"H₀: μ_diferencia = 0 (No hay cambio)")
print(f"H₁: μ_diferencia > 0 (Hubo mejora) [unilateral]")
print(f"α = {alpha}")
print("="*60)

print(f"\nEstadísticos:")
print(f"Media ANTES:      {df_pareado['antes'].mean():.4f}")
print(f"Media DESPUÉS:    {df_pareado['despues'].mean():.4f}")
print(f"Media DIFERENCIA: {df_pareado['diferencia'].mean():.4f}")
print(f"Desv. Std DIFERENCIA: {df_pareado['diferencia'].std():.4f}")

In [ ]:
# Realizar prueba t pareada
t_pareado, p_pareado_bilateral = stats.ttest_rel(df_pareado['despues'], df_pareado['antes'])

# Para prueba unilateral (cola derecha), dividir p-value entre 2
p_pareado_unilateral = p_pareado_bilateral / 2

print("\n" + "="*60)
print("RESULTADOS DE LA PRUEBA")
print("="*60)
print(f"Estadístico t: {t_pareado:.4f}")
print(f"p-value (bilateral): {p_pareado_bilateral:.4f}")
print(f"p-value (unilateral derecha): {p_pareado_unilateral:.4f}")
print("="*60)

In [ ]:
# Decisión (usando p-value unilateral)
print("\n📊 DECISIÓN:")
if p_pareado_unilateral < alpha:
    print(f"   p-value ({p_pareado_unilateral:.4f}) < α ({alpha})")
    print("   ✗ RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: La intervención SÍ tuvo efecto positivo")
    print(f"   Mejora promedio: {df_pareado['diferencia'].mean():.2f} puntos")
    print(f"   ({df_pareado['antes'].mean():.2f} → {df_pareado['despues'].mean():.2f})")
else:
    print(f"   p-value ({p_pareado_unilateral:.4f}) ≥ α ({alpha})")
    print("   ✓ NO RECHAZAMOS H₀")
    print(f"\n💡 CONCLUSIÓN: No hay evidencia suficiente de mejora")

In [ ]:
# Visualización de datos pareados
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de líneas pareadas
for i in range(len(df_pareado)):
    axes[0].plot([1, 2], [df_pareado.iloc[i]['antes'], df_pareado.iloc[i]['despues']], 
                 'o-', color='gray', alpha=0.3)
axes[0].plot([1, 2], [df_pareado['antes'].mean(), df_pareado['despues'].mean()], 
             'ro-', linewidth=3, markersize=10, label='Promedio')
axes[0].set_xticks([1, 2])
axes[0].set_xticklabels(['Antes', 'Después'])
axes[0].set_ylabel('Satisfacción', fontsize=12)
axes[0].set_title('Cambio Individual Antes-Después', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Histograma de diferencias
axes[1].hist(df_pareado['diferencia'], bins=10, color='green', edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Sin cambio')
axes[1].axvline(df_pareado['diferencia'].mean(), color='blue', linestyle='--', linewidth=2, 
               label=f'Media = {df_pareado["diferencia"].mean():.2f}')
axes[1].set_xlabel('Diferencia (Después - Antes)', fontsize=12)
axes[1].set_ylabel('Frecuencia', fontsize=12)
axes[1].set_title('Distribución de Diferencias', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Comparación de Múltiples Áreas (Todas vs Todas)

Si queremos comparar las **4 áreas** simultáneamente, necesitaremos ANOVA (Semana 2). 
Por ahora, veamos un resumen de todas las comparaciones dos a dos:

In [ ]:
# Obtener todas las áreas únicas
areas = df['area'].unique()
print(f"Áreas disponibles: {areas}")

# Crear tabla de comparaciones
from itertools import combinations

comparaciones = []
for area1, area2 in combinations(areas, 2):
    grupo1 = df[df['area'] == area1]['satisfaccion']
    grupo2 = df[df['area'] == area2]['satisfaccion']
    
    t_stat, p_val = stats.ttest_ind(grupo1, grupo2)
    
    comparaciones.append({
        'Área 1': area1,
        'Media 1': grupo1.mean(),
        'Área 2': area2,
        'Media 2': grupo2.mean(),
        'Diferencia': grupo1.mean() - grupo2.mean(),
        't-stat': t_stat,
        'p-value': p_val,
        'Significativo (α=0.05)': '✓' if p_val < 0.05 else '✗'
    })

df_comparaciones = pd.DataFrame(comparaciones)
print("\nCOMPARACIONES ENTRE TODAS LAS ÁREAS")
print("="*100)
print(df_comparaciones.to_string(index=False))

## 7. Reporte Ejecutivo de Pruebas de Hipótesis

In [ ]:
print("="*70)
print("REPORTE EJECUTIVO - PRUEBAS DE HIPÓTESIS")
print("="*70)

print("\n🔬 PRUEBA 1: Satisfacción vs Estándar (8.0)")
print(f"   Resultado: {'Diferente de 8.0' if p_value < 0.05 else 'No diferente de 8.0'}")
print(f"   Media observada: {media_muestral:.2f}")
print(f"   p-value: {p_value:.4f}")

print("\n🔬 PRUEBA 2: Norte vs Sur")
print(f"   Resultado: {'Diferencia significativa' if p_val < 0.05 else 'Sin diferencia significativa'}")
print(f"   Norte: {area_norte.mean():.2f} | Sur: {area_sur.mean():.2f}")
print(f"   p-value: {p_val:.4f}")
print(f"   Tamaño del efecto (d): {d:.2f}")

print("\n🔬 PRUEBA 3: Antes vs Después (Intervención)")
print(f"   Resultado: {'Mejora significativa' if p_pareado_unilateral < 0.05 else 'Sin mejora significativa'}")
print(f"   Antes: {df_pareado['antes'].mean():.2f} | Después: {df_pareado['despues'].mean():.2f}")
print(f"   Mejora: +{df_pareado['diferencia'].mean():.2f} puntos")
print(f"   p-value: {p_pareado_unilateral:.4f}")

print("\n" + "="*70)
print("💡 RECOMENDACIONES:")
print("   1. Investigar causas de diferencias entre áreas")
print("   2. Replicar buenas prácticas del área con mejor desempeño")
print("   3. Continuar monitoreando el impacto de intervenciones")
print("="*70)

## 8. Ejercicios Propuestos

### Ejercicio 1: Calidad de Atención
La ONG afirma que su calidad de atención promedio es de 8.5/10. Realiza una prueba de hipótesis para verificarlo.

### Ejercicio 2: Comparación por Género
¿Existe diferencia significativa en satisfacción entre hombres y mujeres?

### Ejercicio 3: Tiempo de Servicio
Divide a los beneficiarios en "nuevos" (<6 meses) y "antiguos" (≥6 meses). ¿Hay diferencia en satisfacción?

### Ejercicio 4: Múltiples Comparaciones
Compara satisfacción entre las áreas Este y Oeste.


In [ ]:
# Tu código aquí


---

## Resumen

En este notebook aprendiste a:
- ✓ Realizar pruebas t para una muestra
- ✓ Realizar pruebas t para dos muestras independientes
- ✓ Realizar pruebas t pareadas
- ✓ Interpretar p-values y tomar decisiones estadísticas
- ✓ Calcular tamaño del efecto (Cohen's d)
- ✓ Verificar supuestos (test de Levene)
- ✓ Aplicar pruebas en contexto de negocio

**Próximo tema:** Prueba Ji-cuadrada, ANOVA y Regresión (Semana 2)
